### Analysis and visualisation of Bartlett results

In [ ]:
import glob
import pickle
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string
import string
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

bartlett = """One night two young men from Egulac went down to the river to hunt seals and while they were there it became foggy and calm. Then they heard war-cries, and they thought: "Maybe this is a war-party". They escaped to the shore, and hid behind a log. Now canoes came up, and they heard the noise of paddles, and saw one canoe coming up to them. There were five men in the canoe, and they said:
"What do you think? We wish to take you along. We are going up the river to make war on the people."
One of the young men said,"I have no arrows."
"Arrows are in the canoe," they said.
"I will not go along. I might be killed. My relatives do not know where I have gone. But you," he said, turning to the other, "may go with them."
So one of the young men went, but the other returned home.
And the warriors went on up the river to a town on the other side of Kalama. The people came down to the water and they began to fight, and many were killed. But presently the young man heard one of the warriors say, "Quick, let us go home: that man has been hit." Now he thought: "Oh, they are ghosts." He did not feel sick, but they said he had been shot.
So the canoes went back to Egulac and the young man went ashore to his house and made a fire. And he told everybody and said: "Behold I accompanied the ghosts, and we went to fight. Many of our fellows were killed, and many of those who attacked us were killed. They said I was hit, and I did not feel sick."
He told it all, and then he became quiet. When the sun rose he fell down. Something black came out of his mouth. His face became contorted. The people jumped up and cried.
He was dead."""

In [ ]:
def load_rumination_data(pth):
    with open(pth, 'rb') as f: 
        d = pickle.load(f)
    return d

In [ ]:
from wordcloud import (WordCloud, get_single_color_func)
import matplotlib.pyplot as plt


class SimpleGroupedColorFunc(object):
    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)

        

def plot_wordclouds_stages(data, exclusion_text, stages=5):
    # Convert the exclusion text into a set of words for faster lookup, stripping punctuation and lowercasing
    exclusion_text_processed = exclusion_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower()
    exclusion_words = set(exclusion_text_processed.split())

    def preprocess_text(text):
        """Preprocess the text by removing punctuation, lowercasing, and excluding specific words."""
        text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower()
        return ' '.join([word for word in text.split()]) #if word not in exclusion_words])

    # Create a figure for the subplots
    fig, axs = plt.subplots(1, stages, figsize=(stages * 5, 5))

    # Adjust layout
    plt.tight_layout(pad=3.0)

    for stage, samples in enumerate(data[0:stages]):
        # Aggregate and preprocess text for the current stage
        aggregated_text = ' '.join([preprocess_text(sample[0:len(bartlett)]) for sample in samples])

        # Generate word cloud
        if len(aggregated_text.split()) > 0:
            wordcloud = WordCloud(width=400, height=400, relative_scaling=0.4, normalize_plurals=False, collocations=False,
                                  max_font_size=60, background_color='white').generate(aggregated_text)

            color_to_words = {'grey': exclusion_words}

        default_color = 'red'
        grouped_color_func = SimpleGroupedColorFunc(color_to_words, default_color)
        wordcloud.recolor(color_func=grouped_color_func)
            
        axs[stage].imshow(wordcloud, interpolation='bilinear')
        axs[stage].axis('off')
        axs[stage].set_title(f'Stage {stage + 1}')

    # Show plot
    plt.savefig('stages_wordcloud.png', dpi=500)
    plt.show()

In [ ]:
plot_wordclouds_stages(load_rumination_data(pth='bartlett_pkls/rumination_shakespeare.pkl'), bartlett, stages=3)


In [ ]:
plot_wordclouds_stages(load_rumination_data(pth='bartlett_pkls/rumination_papers.pkl'), bartlett, stages=3)


In [ ]:
plot_wordclouds_stages(load_rumination_data(pth='bartlett_pkls/rumination_news.pkl'), bartlett, stages=3)
